https://www.codexa.net/feature-selection-methods/

# continuousな特徴量で、categoricalな目的変数のときの特徴量選択
- Filter method...LDA(フィッシャーのLDA, linear discriminant analysis)
- Wrapper Method...Forward Selection, Backward Elimination, Recursive Feature Elimination
- Embedded Method...Lasso回帰, Ridge回帰, Regularized trees, Memetic algorithm, Random multinomial logit
  - 決定木で特徴の重要度を抽出できたり、線形判別なら係数が直接特徴の重要度となるので、これを用いて変数選択もできそうです。 また、この方法のことをモデルベース特徴量選択というみたいです。



## Filter Method
- 単変量統計 (univariance statistics)
- 統計のテクニックを用いて各特徴の「予測に使える度合」を点数化します。点数をもとに特徴にランク付けを行い、予測に使うか否かをそれぞれ決定していきます。統計的手法で特徴をの有用性を評価する
- 説明変数と目的変数の関係性から変数選択をする。
- 早い
- 最適な特徴の部分集合を見つけられない
- カイ二乗検定（Chi-Square), ANOVA（Analytics of Variance）


## Wrapper Method
- 反復選択 (iterative selection)
- 複数の特徴を同時に使って予測精度の検証を行い、精度が最も高くなるような特徴量の組み合わせを探索していきます。様々な組み合わせでそれぞれ学習を行わせ、その学習結果をもとに組み合わせに優劣をつけていきます。
- 計算量が多い、超遅い
- 交差検証を使用する。(Wrapper Methodを使うときは交差検証しろよ！ということかなと思いました。)
- 結構良い特徴の部分集合が見つかる。
- 選んだ特徴量が過学習する
  - 前進法 (Forward selection)
  - 後退法 (Backward elimination)
  - Recursive Feature Elimination (RFE)


## Embedded Method
- モデルベース特徴量選択（model-based selection）
- 学習アルゴリズムに特徴量選択が組み込まれているので、学習と特徴量選択を同時に行うことができます。
- ラッソ回帰（LASSO Linear Regression）, 決定木 (決定木ベースモデルのfeature_importances_), 線形回帰におけるL1正則化
- lasso
  - 選んだ特徴量が過学習する
  - ロジスティクス回帰やSVMで用いたことしかないので、めっちゃ効く手法もあるか
- ランダムフォレスト
  - どれぐらいの特徴量重要度があったら重要だと言えるのかがイマイチ
  - ランダム性から訓練するたびに特徴量重要度が変動する

# 特徴量選択とコード
- その一　Filter method
https://aotamasaki.hatenablog.com/entry/2018/04/27/222536

```
#mask_oriの計算（以降のエネルギー計算の基準となる）
#ori_y == yとなる場合だけ、E(y)==0となる仕様

selector = SelectKBest(k=num_select) #Filter method #単変量選択（分類：chi2、f_classif、mutual_info_classif）
selector.fit(X, series_y)
mask_ori = selector.get_support() #各特徴量を選択したか否かのmaskを取得
```

- その二　Wrapper method
```
#recursive feature eliminationを用いた選択
#スコア算出ができるので特徴量数を指定できるかもしれない
select = RFECV(LogisticRegression(), cv=10, scoring='average_precision')
select.fit(X, y)
mask = select.support_
```

- その三　Boruta：randomForestを用いた変数選択法
https://www.google.co.jp/url?sa=t&rct=j&q=&esrc=s&source=web&cd=1&ved=0ahUKEwiEzMP4p9naAhWBk5QKHbRjC9oQFggoMAA&url=https%3A%2F%2Fwww.jstatsoft.org%2Farticle%2Fview%2Fv036i11%2Fv36i11.pdf&usg=AOvVaw3tyiHN0BCe2fkkAA6xEVDE

```
forest = RandomForestClassifier()
# define Boruta feature selection method
select = boruta_py.BorutaPy(forest, n_estimators=10)
select.fit(X, y)
mask = select.support_
```